<a href="https://colab.research.google.com/github/Fr33zz/test_projects/blob/main/Joompay_daily_banking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Условия

- Вы владелец магазина. 
- Ваша задача максимизировать прибыль. 
- У вас есть сеть магазинов и данные продаж за день (количество продаж не имеет ярко выраженной сезонности или зависимости от дня недели). 
- К вашим расходам относится покупка товаров и арендная плата за магазины. 
- К доходам - цены от продажи товаров. 

Какие вы можете сделать гипотезы исходя из приведенных данных. Каковы ваши рекомендации?

Подкрепите рекомендации дэшбордом с таблицами или графиками, из которых понятно, как вы получили эти выводы. 

! Выложите документ/документы с тестовым открытыми на просмотр по ссылке

In [2]:
import gspread
from google.auth import default
creds, _ = default()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [4]:
# достаем данные из таблицы 
# (не стал заморачиваться со ссылками и просто скачал)

gc = gspread.authorize(creds)
sp = gc.open('Joompay daily banking')

trn = pd.DataFrame(sp.get_worksheet(1).get_all_values())
trn = trn.set_axis(trn.iloc[0].values, axis='columns')
trn = trn.iloc[1:].astype(int)

gds = pd.DataFrame(sp.get_worksheet(2).get_all_values())
gds = gds.set_axis(gds.iloc[0].values, axis='columns')
gds = gds.iloc[1:].astype(int)

mag = pd.DataFrame(sp.get_worksheet(3).get_all_values())
mag = mag.set_axis(mag.iloc[0].values, axis='columns')
mag = mag.iloc[1:].astype(int)

RefreshError: ignored

In [ ]:
# смотрим на таблицу с транзакциями
# меняем названия для удобства
# подсчитываем индексы для поиска совпадений пар и троек

trn.columns = ['id', 'client_id', 'gds_id', 'count', 'mag_id']
trn.head(10)

In [ ]:
# смотрим на таблицу с товарами (goods)
# сразу считаю прибыль на товар и маржу

gds.columns = ['gds_id', 'price', 'cost']
gds['profit_'] = gds['price'] - gds['cost']
gds['margin'] = gds['profit_'] / gds['cost']
gds

In [ ]:
# смотрим на таблицу с магазинами
mag.columns = ['mag_id', 'rent']
mag

In [ ]:
# делаю join таблиц в одну
d = pd.merge(left = trn, right = gds, how = 'left', on= 'gds_id')
d = pd.merge(left =   d, right = mag, how = 'left', on= 'mag_id')
d['revenue'] = d['price'] * d['count']
d['expense'] = d['cost'] * d['count']
d

###На этот бизнес можно смотреть с 3 разных сторон

- **управление магазинами (сетевик)**
 - рост числа магазинов
 - увеличение прибыли
 - посмотреть методики от сетей
 - подумать о продуктовых, о е-коме (доставка), о питании
- **дистрибуция товаров**
 - оптимизация затрат на аренду
 - добавление маржинальных и уменьшение немаржинальных
 - фишки ритейла (бакалея и ко)
- **управление клиентской базой (e-com)**
 - привлечение новых платящих клиентов
 - повышение чека или числа покупок текущих клиентов
 - другие способы монетизации аудитории

####**вопросы на которые далее интересно получить ответы**

- [ ] прибыльность продуктов с учетом аренды
- [ ] какова клиентская база
- [ ] есть ли постоянные клиенты
- [ ] есть ли любимые товары
- [ ] есть ли пересечения базы у магазинов (находятся ли они рядом)
- [ ] какой допустимый CAC пользователя
- [ ] сколько уникальных клиентов у магазина
- [ ] какие магазины стоит подключать
- [ ] какое число уникальных комбинаций клиент-товар-магазин, есть ли повторы
- [ ] можно ли перебрасывать аудиторию из одних магазинов в другие
- [ ] какую модель можно здесь предложить

In [ ]:
# Посчитаем экономику магазина

mag = \
pd.merge(left = mag,
         right = d.groupby('mag_id')\
                  .agg({'revenue':'sum', 'expense':'sum', 'count':'sum'})\
                  .reset_index(),
         how = 'left', on = 'mag_id')

# pgs = per good sold

mag['profit'] = mag['revenue'] - mag['expense'] - mag['rent']
mag['mag_margin'] = mag['profit'] / (mag['expense'] + mag['rent'])
mag['profit_pgs'] = mag['profit'] / mag['count']
mag['expense_pgs'] = mag['expense'] / mag['count']
mag['rent_pgs'] = mag['rent'] / mag['count']
mag

### Гипотезы увеличения прибыли магазинов
Интересный факт – второй самый немаржинальный, при этом у него самая высокая прибыль на единицу проданных товаров
- подключить больше магазинов 2 типа как самых прибыльных

явно виден гэп между маржой и профитом в 1 и 2 магазинах
- завезти в 1 магазин больше товаров как во 2

видно что по 2 и 3 продают ±одинаково, а оборот оч разный – можно перераспределить товары или продавать больше или повысить средний чек

In [ ]:
# Посчитаем экономику каждого вида товаров

d['rent_pgs'] = d['mag_id'].map(mag[['mag_id','rent_pgs']].set_index('mag_id').to_dict()['rent_pgs'])
d['rent_exp'] = d['rent_pgs'] * d['count']
d['profit_final'] = d['revenue'] - d['expense'] - d['rent_exp']

gds = \
pd.merge(left = gds,
         right = d.groupby('gds_id')\
                  .agg({'count':'sum', 'rent_pgs':['min','mean','max']})\
                  .reset_index(),
         how = 'left', on = 'gds_id'
)
gds.columns = ['gds_id', 'price','cost', 'profit_', 'margin', 'count', 'rent_min', 'rent_mean', 'rent_max']

gds['actual_profit'] = gds['profit_'] - gds['rent_mean']
gds['profit_min'] = gds['profit_'] - gds['rent_max']
gds['profit_max'] = gds['profit_'] - gds['rent_min']

gds

###Гипотезы увеличения прибыли с товаров
- во первых надо точно перевезти все товары первой категории в другой стор, тк сейчас они убыточны
- если клиенты двигают за своим товаром в любой магазин, то просто перетасовать товары (решается простая задача оптимизации, даже в случае тысяч магазинов)

в нашем случае скорее всего придется ставить задачу в стиле "у каждого магазина свой пул клиентов, они могут купить вот то и это, надо закупить товар так чтобы оптимизировать прибыль при фиксированном уровне расходов"

In [ ]:
# Посчитаем экономику клиента
cln = \
d.groupby('client_id').agg({'gds_id':lambda x: np.unique(x), 
                            'mag_id':lambda x: np.unique(x), 
                            'count':'sum',
                            'price':['min', 'mean','max'],
                            'revenue':'sum',
                            'expense':'sum',
                            'rent_exp':'sum'})

cln.columns = ['gds_id', 'mag_id', 'count', 
               'price min', 'price mean', 'price max', 
               'revenue', 'expense', 'rent_exp']

cln['profit'] = cln['revenue'] - cln['expense'] - cln['rent_exp']
cln

### Гипотезы по улучшению клиентской базы
- закрыть позиции товаров в магазинах в которые приходят не прибыльные клиенты
- кластеризовать клиентов по выручке и изучить их опыт, чтобы привлекать больше таких
- первый товар перевести в другой магазин (с меньшей рентой)

In [ ]:
d.groupby(['client_id','gds_id']).agg({'profit_final':'sum'}).unstack().fillna(' ')

In [ ]:
d.groupby(['client_id','mag_id']).agg({'profit_final':'sum'}).unstack().fillna(' ')

In [ ]:
d.groupby(['mag_id','gds_id']).agg({'profit_final':'sum'}).unstack().fillna(' ')